### Average Distance Check

In [20]:
from ase.io import read
import numpy as np
from tkinter.filedialog import askopenfilename

# 거리 조건 설정 (단위: Å)
min_cutoff = 1.7
max_cutoff = 2.3

# 구조 파일 선택
file_path = askopenfilename(title="Select structure file",
                            filetypes=[("Structure files", "*.cif *.vasp *POSCAR* *CONTCAR*")])

atoms = read(file_path)

# Ni, O 원자 인덱스 추출
ni_indices = [i for i, atom in enumerate(atoms) if atom.symbol == 'Ni']
fe_indices = [i for i, atom in enumerate(atoms) if atom.symbol == 'Fe']
o_indices  = [i for i, atom in enumerate(atoms) if atom.symbol == 'O']

# 거리 계산 (min_cutoff ≤ distance ≤ max_cutoff)
distances_1 = []
Ni_each = []
Fe_each = []
for i in ni_indices:
    ni_temp = []
    for j in o_indices:
        d = atoms.get_distance(i, j, mic=True)
        if min_cutoff <= d <= max_cutoff:
            distances_1.append(d)
            ni_temp.append(d)
    Ni_each.append(np.mean(ni_temp))


distances_2 = []
for i in fe_indices:
    fe_temp = []
    for j in o_indices:
        d = atoms.get_distance(i, j, mic=True)
        if min_cutoff <= d <= max_cutoff:
            distances_2.append(d)
            fe_temp.append(d)
    Fe_each.append(np.mean(fe_temp))

# 출력
if distances_1:
    mean_distance_1 = np.mean(distances_1)
    mean_distance_2 = np.mean(distances_2)
    print(f"Average Ni–O distance within {min_cutoff}-{max_cutoff} Å: {mean_distance_1:.2f} Å")
    print(f"Average Fe–O distance within {min_cutoff}-{max_cutoff} Å: {mean_distance_2:.2f} Å")
    print(f"Number of valid Ni–O pairs: {len(distances_1)}")
    print(f"Number of valid Fe–O pairs: {len(distances_2)}")
else:
    print(f"No Ni–O pairs found within the {min_cutoff}-{max_cutoff} Å range.")

for i in Ni_each :
    print(f"Ni : {i:.2f}")

for i in Fe_each :
    print(f"Fe : {i:.2f}")

Average Ni–O distance within 1.7-2.3 Å: 1.87 Å
Average Fe–O distance within 1.7-2.3 Å: 1.99 Å
Number of valid Ni–O pairs: 34
Number of valid Fe–O pairs: 11
Ni : 1.87
Ni : 1.88
Ni : 1.86
Ni : 1.87
Ni : 1.87
Ni : 1.88
Fe : 1.95
Fe : 2.02


In [5]:
print(distances_1)

[2.085309238331063, 1.8836045200195304, 2.0879961116028976, 1.8793838616031622, 1.8806852149052273, 1.8961376847803935, 1.8181942726227684, 1.8708725482925577, 1.879143945611613, 1.8583767115941205, 1.8668467509528073, 1.8546532918351528, 1.8613467260742274, 1.8480288320220846, 1.8717508965906564, 1.8518006499192619, 1.9951559419401772, 1.880267278930294, 1.8694854500559703, 1.8420493160857303, 1.8464268654846292, 1.8637444912433077, 1.8596519334440345, 1.9569700261861176, 1.8870308404917744, 1.827137137114256, 1.8529505194606493, 1.854050896827922, 1.8641887972100986, 1.864584048398396, 1.867150760415062, 1.874322357393964, 1.8584439631413703, 1.875081922402603, 1.9511785296235638, 1.8320910667808261, 1.8498017714448793, 1.8889217349473166, 1.8556000978979534, 1.8717086075642282, 1.86918012540842, 1.8463948857029873, 1.8526506401299345, 1.860189587687038, 1.8647368573852223]


### 폴더 선택 후 해당 폴더에 존재하는 모든 OSZICAR 결과 출력


In [4]:
import os
import pandas as pd
from tkinter import filedialog, Tk


def find_last_F_line(oszicar_path):
    try:
        with open(oszicar_path, 'r') as f:
            lines_with_F = [line.strip() for line in f if 'F' in line]
        return lines_with_F[-1] if lines_with_F else None
    except Exception as e:
        return f"Error reading file: {e}"

def search_oszicar_in_folders(root_dir):
    data = []
    for dirpath, _, filenames in os.walk(root_dir):
        if 'OSZICAR' in filenames:
            oszicar_path = os.path.join(dirpath, 'OSZICAR')
            last_line = find_last_F_line(oszicar_path)
            if last_line:
                rel_path = os.path.relpath(dirpath, root_dir)
                data.append((rel_path, last_line))
    df = pd.DataFrame(data, columns=["Folder", "Last F Line"])

    split_cols = df["Last F Line"].str.split(expand=True)
    split_cols.columns = [f"Field_{i}" for i in range(split_cols.shape[1])]

    selected = pd.concat([df["Folder"], split_cols[["Field_3", "Field_4"]]], axis=1)
    return selected



Tk().withdraw()
root_folder = filedialog.askdirectory(title="OSZICAR 파일을 찾을 루트 폴더 선택")
if not root_folder:
    print("선택된 폴더가 없습니다.")
    exit()

df = search_oszicar_in_folders(root_folder)
if df.empty:
    print("F가 포함된 OSZICAR 라인이 없습니다.")
else:
    display(df)  # Jupyter 환경에서 표로 출력

home_dir = os.path.join(os.path.expanduser("~"), "Desktop")
save_path = os.path.join(home_dir, "oszicar_results.csv")

df.to_csv(save_path, index=False, encoding='utf-8-sig')

,Folder,Field_3,Field_4
0,01,E0=,-.60038942E+03
1,02,E0=,-.60053964E+03


In [8]:
save_path = filedialog.asksaveasfilename(defaultextension=".csv",
                                        filetypes=[("CSV files", "*.csv")],
                                        title="Save OSZICAR summary as CSV")

df.to_csv(save_path, index=False, encoding='utf-8-sig')

NameError: name 'df' is not defined